Data from [2018 Behavioral Risk Factor Surveillance System survey](Ahttps://www.cdc.gov/brfss/annual_data/annual_2018.html).

The raw data are fairly large - about 100M zip compressed, and nearly 1G when uncompressed.

Data from the [SAS transport format download[(https://www.cdc.gov/brfss/annual_data/2018/files/LLCP2018XPT.zip).

Variable list in the [code book](https://www.cdc.gov/brfss/annual_data/2018/pdf/codebook18_llcp-v2-508.pdf).

In [1]:
import os
import os.path as op
import urllib.request
from zipfile import ZipFile

In [2]:
# Fetch data file from URL.
brfss_url = 'https://www.cdc.gov/brfss/annual_data/2018/files/LLCP2018XPT.zip'
zfname, msg = urllib.request.urlretrieve(brfss_url, 'LLCP2018XPT.zip')

In [3]:
# Get data file.
with ZipFile(zfname) as zf:
    zf.extract('LLCP2018.XPT ')
# Remove trailing space in file extension in order for Pandas to recognize
# the `.xpt` extension as SAS transport format.
os.rename('LLCP2018.XPT ', 'LLCP2018.xpt')

In [4]:
import numpy as np
import pandas as pd

In [5]:
df = pd.read_sas('LLCP2018.xpt')
df.head()

,_STATE,FMONTH,IDATE,IMONTH,IDAY,IYEAR,DISPCODE,SEQNO,_PSU,CTELENM1,...,_MAM5022,_RFPAP34,_RFPSA22,_RFBLDS3,_COL10YR,_HFOB3YR,_FS5YR,_FOBTFS,_CRCREC,_AIDTST3
0,1.0,1.0,b'01052018',b'01',b'05',b'2018',1100.0,b'2018000001',2.018000e+09,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
1,1.0,1.0,b'01122018',b'01',b'12',b'2018',1100.0,b'2018000002',2.018000e+09,1.0,...,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
2,1.0,1.0,b'01082018',b'01',b'08',b'2018',1100.0,b'2018000003',2.018000e+09,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0
3,1.0,1.0,b'01032018',b'01',b'03',b'2018',1100.0,b'2018000004',2.018000e+09,1.0,...,NaN,NaN,2.0,2.0,2.0,2.0,2.0,2.0,2.0,2.0
4,1.0,1.0,b'01122018',b'01',b'12',b'2018',1100.0,b'2018000005',2.018000e+09,1.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.0


There are about half a million rows, and several hundred columns.

In [6]:
df.shape

(437436, 275)

`_STATE` is [State FIPS Code](https://en.wikipedia.org/wiki/Federal_Information_Processing_Standard_state_code).

In [7]:
state_fips = df['_STATE'].astype(int)
state_fips.value_counts(sort=False)

1      6606
2      2758
4      8089
5      5360
6     11919
8      8899
9     10709
10     5233
11     4397
12    15242
13     9305
15     7901
16     3696
17     5304
18     7608
19     9127
20    10822
21     8548
22     5125
23    10971
24    17546
25     6669
26    10322
27    16990
28     5843
29     6231
30     5190
31    14634
32     3222
33     5737
34     3090
35     6713
36    35767
37     4729
38     5622
39    12763
40     5253
41     5946
42     6213
44     5607
45    10843
46     7120
47     5160
48    11217
49    10563
50     6544
51    10321
53    13120
54     4897
55     4996
56     4462
66     1644
72     4843
Name: _STATE, dtype: int64

Replace with two-letter postal abbreviation:

In [8]:
us_states = pd.read_csv(op.join('..', 'us_states', 'us_states.csv'))
us_states.head()

,fips,post_code,name,gnisid
0,1,AL,Alabama,1779775
1,2,AK,Alaska,1785533
2,4,AZ,Arizona,1779777
3,5,AR,Arkansas,68085
4,6,CA,California,1779778


In [9]:
# Make a Series to look up state abbreviations from FIPs codes.
lookup = us_states['post_code']
lookup.index = us_states['fips']
lookup.head()

fips
1    AL
2    AK
4    AZ
5    AR
6    CA
Name: post_code, dtype: object

In [10]:
state_fips.head()

0    1
1    1
2    1
3    1
4    1
Name: _STATE, dtype: int64

In [11]:
# Look up abbreviations from FIPS code.
state_pcs = lookup.loc[state_fips]
# Set original index.
state_pcs.index = state_fips.index
state_pcs.value_counts(sort=False)

DC     4397
GU     1644
UT    10563
AK     2758
ND     5622
NE    14634
CA    11919
CO     8899
NM     6713
NH     5737
MA     6669
LA     5125
OH    12763
CT    10709
IN     7608
PA     6213
PR     4843
TX    11217
NJ     3090
WI     4996
AZ     8089
WV     4897
DE     5233
ME    10971
NV     3222
KY     8548
HI     7901
OR     5946
MD    17546
RI     5607
MN    16990
TN     5160
GA     9305
VA    10321
WA    13120
SC    10843
AR     5360
NY    35767
MT     5190
NC     4729
VT     6544
AL     6606
MS     5843
MI    10322
KS    10822
MO     6231
IA     9127
FL    15242
ID     3696
WY     4462
SD     7120
IL     5304
OK     5253
Name: post_code, dtype: int64

`QSTVER`: "Questionnaire Version Identifier".

* 10 Landline single questionnaire
* 11 Landline multiple questionnaire version 1
* 12 Landline multiple questionnaire version 2
* 13 Landline multiple questionnaire version 3
* 20 Cell Phone single questionnaire
* 21 Cell Phone multiple questionnaire version 1
* 22 Cell Phone multiple questionnaire version 2
* 23 Cell Phone multiple questionnaire version 3

In [12]:
qstver = df['QSTVER']
qstver.value_counts()

20.0    163876
10.0     93507
21.0     47939
22.0     45491
11.0     32777
12.0     29197
23.0     14848
13.0      9801
Name: QSTVER, dtype: int64

In [13]:
qstver.head()

0    10.0
1    10.0
2    10.0
3    10.0
4    10.0
Name: QSTVER, dtype: float64

In [14]:
def recode_qt(v):
    if v < 20:
        return "LAND"
    if v >= 20:
        return 'CELL'
    return v

qst_type = qstver.apply(recode_qt)
qst_type.value_counts()

CELL    272154
LAND    165282
Name: QSTVER, dtype: int64

In [15]:
qst_type.head()

0    LAND
1    LAND
2    LAND
3    LAND
4    LAND
Name: QSTVER, dtype: object

`QSTLANG`: "Language identifier"

* 1 English
* 2 Spanish
* 3-99 Other

In [16]:
qstlang = df['QSTLANG']
qstlang.value_counts()

1.0    421830
2.0     15541
3.0        17
Name: QSTLANG, dtype: int64

In [17]:
def recode_ql(v):
    if v == 1:
        return 'ENG'
    if v == 2:
        return 'SPA'
    return np.nan

qstlang_labels = qstlang.apply(recode_ql)
qstlang_labels.value_counts()

ENG    421830
SPA     15541
Name: QSTLANG, dtype: int64

`_URBSTAT` : "Urban/Rural Status"

* 1 Urban counties 
* 2 Rural counties

In [18]:
urbstat = df['_URBSTAT']
urbstat.value_counts()

1.0    365714
2.0     65235
Name: _URBSTAT, dtype: int64

In [19]:
def recode_1_2(v):
    if v == 1:
        return 'URBAN'
    if v == 2:
        return 'RURAL'
    return v

urbstat_labels = urbstat.apply(recode_1_2)
urbstat_labels.value_counts()

URBAN    365714
RURAL     65235
Name: _URBSTAT, dtype: int64

`HTM4` is the "Computed Height in Meters", with 2 implied decimal places, so in fact it is the computed height in centimeters.

It is computed because the original variable, `HEIGHT3` could be in feet / inches or meters.

In [20]:
height_cm = df['HTM4']
height_cm.head()

0    163.0
1    165.0
2    147.0
3    178.0
4    160.0
Name: HTM4, dtype: float64

In [21]:
# Heights are all integers.
good_height = height_cm.dropna()
np.all(good_height == np.round(good_height))

True

`WTKG3` is "Computed Weight in Kilograms".  It has 2 implied decimal places, so it is in units of 10 grams.  It is computed from the mixed unit `WEIGHT2` variable.

In [22]:
weight_kg = df['WTKG3'] / 100
weight_kg.head()

0    58.97
1    90.72
2    64.41
3    86.18
4      NaN
Name: WTKG3, dtype: float64

`GENHLTH` is "General Health":

* 1 Excellent
* 2 Very good
* 3 Good
* 4 Fair
* 5 Poor
* 7 Don’t know/Not Sure
* 9 Refused
* BLANK Not asked or Missing

In [23]:
genhlth = df['GENHLTH']
genhlth.value_counts()

2.0    142197
3.0    138321
1.0     71893
4.0     60762
5.0     23120
7.0       800
9.0       318
Name: GENHLTH, dtype: int64

In [24]:
def recode_7_9(val):
    if val in (7, 9):
        return np.nan
    return val

In [25]:
genhlth_clean = genhlth.apply(recode_7_9)
genhlth_clean.value_counts()

2.0    142197
3.0    138321
1.0     71893
4.0     60762
5.0     23120
Name: GENHLTH, dtype: int64

`PHYSHLTH`: "Number of Days Physical Health Not Good"; "Now thinking about your physical health, which includes physical illness and injury, for how many days during the past 30 days was your physical health not good?".

* 1 - 30 Number of days
* 88 None
* 77 Don’t know/Not sure
* 99 Refused
* BLANK Not asked or Missing

In [26]:
physhlth = df['PHYSHLTH']
physhlth.describe()

count    437410.000000
mean         60.122526
std          37.156196
min           1.000000
25%          15.000000
50%          88.000000
75%          88.000000
max          99.000000
Name: PHYSHLTH, dtype: float64

In [27]:
def recode_88_77_99(val):
    if val == 88:
        return 0
    if val in (77, 99):
        return np.nan
    return val

In [28]:
physhlth_clean = physhlth.apply(recode_88_77_99)
physhlth_clean.describe()

count    428472.000000
mean          4.424597
std           8.931486
min           0.000000
25%           0.000000
50%           0.000000
75%           3.000000
max          30.000000
Name: PHYSHLTH, dtype: float64

`MENTHLTH`: "Number of Days Mental Health Not Good" ; "Now thinking about your mental health, which includes stress, depression, and problems with emotions, for how many days during the past 30 days was your mental health not good?".

* 1 - 30 Number of days
* 88 None
* 77 Don’t know/Not sure
* 99 Refused
* BLANK Not asked or Missing

In [29]:
menhlth = df['MENTHLTH']
menhlth.describe()

count    437411.000000
mean         63.066830
std          36.385937
min           1.000000
25%          20.000000
50%          88.000000
75%          88.000000
max          99.000000
Name: MENTHLTH, dtype: float64

In [30]:
menhlth_clean = menhlth.apply(recode_88_77_99)
menhlth_clean.describe()

count    430423.000000
mean          3.683923
std           7.984771
min           0.000000
25%           0.000000
50%           0.000000
75%           2.000000
max          30.000000
Name: MENTHLTH, dtype: float64

`SLEPTIM1`: "How Much Time Do You Sleep" ; "On average, how many hours of sleep do you get in a 24-hour period?"

* 1 - 24 Number of hours
* 77 Don’t know/Not sure
* 99 Refused
* BLANK Missing

In [31]:
hrs_sleep = df['SLEPTIM1']
hrs_sleep.describe()

count    437413.000000
mean          7.905229
std           8.052407
min           1.000000
25%           6.000000
50%           7.000000
75%           8.000000
max          99.000000
Name: SLEPTIM1, dtype: float64

In [32]:
hrs_sleep_clean = hrs_sleep.apply(recode_88_77_99)
hrs_sleep_clean.describe()

count    432099.000000
mean          7.031259
std           1.510051
min           1.000000
25%           6.000000
50%           7.000000
75%           8.000000
max          24.000000
Name: SLEPTIM1, dtype: float64

`SEX1` : "Respondents Sex" ; "What is your sex? or What was your sex at birth? Was it..."

* 1 Male
* 2 Female
* 7 Don't know/Not Sure
* 9 Refused

In [33]:
sex = df['SEX1']
sex.value_counts()

2.0    238911
1.0    197412
9.0       682
7.0       431
Name: SEX1, dtype: int64

In [34]:
sex_clean = sex.apply(recode_7_9)
sex_clean.value_counts()

2.0    238911
1.0    197412
Name: SEX1, dtype: int64

In [35]:
sex_labels = sex_clean.astype(object)
sex_labels[sex_clean == 1] = 'M'
sex_labels[sex_clean == 2] = 'F'
sex_labels.value_counts()

F    238911
M    197412
Name: SEX1, dtype: int64

`SMOKE100` : "Smoked at Least 100 Cigarettes" ; "Have you smoked at least 100 cigarettes in your entire life?"

* 1 Yes
* 2 No
* 7 Don't know/Not Sure
* 9 Refused
* BLANK Not asked or Missing

In [36]:
smoke100 = df['SMOKE100']
smoke100.value_counts()

2.0    240594
1.0    180436
7.0      2104
9.0       458
Name: SMOKE100, dtype: int64

In [37]:
smoke100_clean = smoke100.apply(recode_7_9)
smoke100_clean.value_counts()

2.0    240594
1.0    180436
Name: SMOKE100, dtype: int64

In [38]:
smoke100_labels = smoke100_clean.astype(object)
smoke100_labels[smoke100_clean == 1] = True
smoke100_labels[smoke100_clean == 2] = False
smoke100_labels.value_counts()

False    240594
True     180436
Name: SMOKE100, dtype: int64

In [39]:
clean_df = pd.DataFrame()
cols = [qst_type, qstlang_labels, urbstat_labels,
        state_pcs, height_cm, weight_kg, sex_clean,
         genhlth_clean, physhlth_clean,
         menhlth_clean, smoke100_clean]
names = ['interview_type', 'quest_lang', 'urban_rural',
         'state', 'height_cm', 'weight_kg',
         'gen_health', 'phys_health_days', 'mental_health_days',
          'smoke100']
for col, name in zip(cols, names):
    clean_df[name] = col

clean_df.head()

,interview_type,quest_lang,urban_rural,state,height_cm,weight_kg,gen_health,phys_health_days,mental_health_days,smoke100
0,LAND,ENG,RURAL,AL,163.0,58.97,2.0,2.0,30.0,0.0
1,LAND,ENG,URBAN,AL,165.0,90.72,2.0,3.0,0.0,0.0
2,LAND,ENG,URBAN,AL,147.0,64.41,2.0,5.0,10.0,0.0
3,LAND,ENG,URBAN,AL,178.0,86.18,1.0,1.0,0.0,0.0
4,LAND,ENG,URBAN,AL,160.0,NaN,2.0,2.0,0.0,0.0


In [40]:
out_fname = op.join('processed', 'brfss_vars.csv')
clean_df.to_csv(out_fname, index=False)
pd.read_csv(out_fname).head()

,interview_type,quest_lang,urban_rural,state,height_cm,weight_kg,gen_health,phys_health_days,mental_health_days,smoke100
0,LAND,ENG,RURAL,AL,163.0,58.97,2.0,2.0,30.0,0.0
1,LAND,ENG,URBAN,AL,165.0,90.72,2.0,3.0,0.0,0.0
2,LAND,ENG,URBAN,AL,147.0,64.41,2.0,5.0,10.0,0.0
3,LAND,ENG,URBAN,AL,178.0,86.18,1.0,1.0,0.0,0.0
4,LAND,ENG,URBAN,AL,160.0,NaN,2.0,2.0,0.0,0.0
